# Automatic Fact-checking project using pandas and ScikitLearn

### Travail réalisé par : Bouali Mohammed-Amin, Oussama Nassim Sehout, Chahinez Benallal, Abdellah Choukri 



####  Sommaire du travail :
####   . Chargement du jeu de données
####   . Prétraitement du texte 
###   . Conversion des textes aux valeurs numériques
###   . Division du jeu de données en données d'entraînement et données de test
###   . Entraînement et prédiction en utilisant des modèles de classification

## Etape 1 : Chargement du jeu de données  
 On a généré trois fichiers csv, un avec valeurs true, un autre avec valeurs false, et un dernier avec valeurs mixture à partir du site ClaimsKg, après on les concatène pour avoir notre jeu de données.

In [1]:
import pandas as pd
import random as random
import glob, os

#concatener les fichiers en python  et lecture du fichier :
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "./DataSet/*.csv"))))

row, col = df.shape
print("Size of the dataframe :" + str(df.size) + " ("+str(row)+"*"+str(col)+")")

df.head()

df

Size of the dataframe :342328 (24452*14)


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/47...,'Hillary's main extracurricular activity in la...,2008-01-24,1,False,Chain email,"She did not """"help"""" the Black Panthers","ACLU,Alex Rackley,American Civil Liberties Uni...",Black Panthers,Candidate Biography,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/1d...,Says 'Hillary Clinton's plan would bring in 62...,2016-09-20,1,False,Donald Trump,"Trump says Clinton would bring in 620,000 refu...","Barack Obama,CBS,Center for Global Development...",Hillary Clinton,Immigration,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
2,http://data.gesis.org/claimskg/claim_review/51...,Canada suffered a major donut shortage shortly...,Unknown,1,False,Unknown,Did Canadians Face a Major Donut Shortage Afte...,"CNN,Daily Bonnet,Girl Scout cookies,Tim Horton...",marijuana,"canada, marijuana, world news daily report",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/canada-donut...,English
3,http://data.gesis.org/claimskg/claim_review/c5...,"The mayor of Ath, in Belgium, refused to remov...",Unknown,1,False,Unknown,Did a Belgian Mayor Refuse to Ban Pork in Scho...,"Belgium,Islam,Maryborough, Victoria,Muslim,Que...",NaN,islam,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/belgium-mayo...,English
4,http://data.gesis.org/claimskg/claim_review/ef...,Says Pete Gallego canceled a 'planned meeting ...,2018-07-18,1,False,Roland Gutierrez,Roland Gutierrez errs saying Pete Gallego canc...,"2012 mass shooting,Carlos Uresti,Democrat,Face...",Pete Gallego,"Congress,Guns",politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,http://data.gesis.org/claimskg/claim_review/d7...,A pedestrian was killed by a flying fire hydrant.,Unknown,3,True,Unknown,Fire Hydrant Death,"Associated Press,Encinitas, California,Fire Hy...",fire hydrant,"ASP Article, freakish fatalities",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/fire-plugged/,English
4448,http://data.gesis.org/claimskg/claim_review/b5...,'The United States is the ONLY industrialized ...,2019-04-30,3,True,Mark Pocan,Universal health care diagnosis is on the mark,"Affordable Care Act,Australia,Austria,Belarus,...",NaN,"Health Care,Medicaid,Medicare",politifact,http://www.politifact.com,http://www.politifact.com/wisconsin/statements...,English
4449,http://data.gesis.org/claimskg/claim_review/05...,A 17-year-old Nigerian girl was publicly flogg...,Unknown,3,True,Unknown,Bariya Ibrahim Magazu Petition,"Abuja,Associated Press,Barbara “faith,Dallas M...",premarital sex,"ASP Article, Petitions",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/the-cane-mut...,English
4450,http://data.gesis.org/claimskg/claim_review/7d...,'We haven’t had control of the House for the l...,2010-10-18,3,True,William G. Batchelder,Republican state Rep. Bill Batchelder knocks D...,"Armond Budish,Democrat,PolitiFact,Republican",NaN,Government Efficiency,politifact,http://www.politifact.com,http://www.politifact.com/ohio/statements/2010...,English


### Etape 2 : Prétraitement du texte
 On a fait les prétraitements suivants :
     . Transformation du texte en miniscule
     . Suppression des espaces
     . Enlever les ponctuations 
     . Elimination des stopwords
     . Remplacement des mots de négation par le mot 'not'
     . Suppression des caractères non ASCII
     . Lemmatization 
     . Correction orthographique ##TODO

In [2]:
#commencement des prétraitement :

#import nécessaire :
import re
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
from nltk.corpus import stopwords
import unicodedata
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


#La j'ai mis que la colonne text en lower mais je crois qu'on devra tous les mettre par la suite
df['text'] = df['text'].str.lower()


#White spaces removal
df['text'] = df['text'].str.strip()


#Elimination des stopWord :
#input_str = "NLTK is a leading platform for building Python programs to work with human language data."
stop_words = set(stopwords.words("english"))
#La liste des stopwords en Anglais
negation ={'haven''t','cannot',"doesn't","shouldn't","needn't","shant't","weren't","hasn't", "wasn't","didn't", "aren't",'not', "mightn't", "mustn't", 'no',  "wouldn't", "mightn't", "won't",  "needn't", "wasn't", "wouldn't",  "isn't", "doesn't", "weren't", "isn't", "hasn't", "hadn't", "don't", "hadn't","couldn't"}

#La liste des stopwords de négation en Anglais  
from nltk.tokenize import TweetTokenizer
a= df['text'].str.replace("’","'") #Extraire toutes les entrées de la colonne text
pat = r'\b(?:{})\b'.format('|'.join(negation))

a=a.str.replace(pat,'not')
pat='\w*\d\w*'
text_without_stopwords=[] #Une liste dont on va affecter les textes après l'élimination des stop words
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
 tokens = tk.tokenize(str(text[1]))
 result = [i for i in tokens if not i in stop_words-negation]
 splitor=" "
 concatinated = splitor.join(result) #concatiner les tokens
 text_without_stopwords.append(concatinated)

df['text']=text_without_stopwords


#The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:
text_without_punctuation=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    result = re.sub('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', '', str(text[1]) )
    splitor="" #séparateur de mots
    concatinated = splitor.join(result) #concatiner les résultats
    text_without_punctuation.append(concatinated)

df['text']=text_without_punctuation


#The following code removes non ASCII characters :
text_without_ascii=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    text = unicodedata.normalize('NFKD', str(text[1]) ).encode("ascii", "ignore").decode("utf-8", 'ignore')
    splitor="" #séparateur de mots
    concatinated = splitor.join(text)
    text_without_ascii.append(concatinated)

df['text']=text_without_ascii


#Lemmatization :
lemmatizer = WordNetLemmatizer()
text_lemmatizer=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for ligne in a.iteritems(): #On parcourt toutes les lignes
    wordList = re.sub("[^\w]", " ",  ligne[1]).split() # on parcourt tout les mot de la ligne
    newList=[]
    for word in wordList:
        if not(word.isdigit()): # si c'est  mot et non un nombre
            newList.append(lemmatizer.lemmatize(word, pos = 'v')) #lemmatisation
    splitor=" " #séparateur de mots
    concatinated = splitor.join(newList)
    text_lemmatizer.append(concatinated)

df['text']=text_lemmatizer
df

#Correction orthographique:
#from textblob import TextBlob ceci ne marche pas encore une installation est requise
#df['text'][:5].apply(lambda x: str(TextBlob(x).correct())) 

#suppression des common word:
from collections import Counter
word_counter  = Counter()
for sentence in df["text"].values:
    for word in sentence.split():
        word_counter[word] += 1
most = word_counter.most_common(10)
print("most common word"+str(most))
print("Suppression of the common word de nos artiles : ")
most_word = set([w for (w, wc) in most])
def delmost_word(sentence):
    return " ".join([word for word in str(sentence).split() if word not in most_word])
df["text"] = df["text"].apply(delmost_word)
df["text"].head()

df

#Fin des prétraitement.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chahinez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chahinez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chahinez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


most common word[('say', 4888), ('s', 3903), ('not', 2872), ('state', 2269), ('president', 1825), ('obama', 1699), ('percent', 1672), ('show', 1558), ('tax', 1509), ('trump', 1433)]
Suppression of the common word de nos artiles : 


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/47...,hillary main extracurricular activity law scho...,2008-01-24,1,False,Chain email,"She did not """"help"""" the Black Panthers","ACLU,Alex Rackley,American Civil Liberties Uni...",Black Panthers,Candidate Biography,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/1d...,hillary clinton plan would bring refugees firs...,2016-09-20,1,False,Donald Trump,"Trump says Clinton would bring in 620,000 refu...","Barack Obama,CBS,Center for Global Development...",Hillary Clinton,Immigration,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
2,http://data.gesis.org/claimskg/claim_review/51...,canada suffer major donut shortage shortly cou...,Unknown,1,False,Unknown,Did Canadians Face a Major Donut Shortage Afte...,"CNN,Daily Bonnet,Girl Scout cookies,Tim Horton...",marijuana,"canada, marijuana, world news daily report",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/canada-donut...,English
3,http://data.gesis.org/claimskg/claim_review/c5...,mayor ath belgium refuse remove pork school ca...,Unknown,1,False,Unknown,Did a Belgian Mayor Refuse to Ban Pork in Scho...,"Belgium,Islam,Maryborough, Victoria,Muslim,Que...",NaN,islam,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/belgium-mayo...,English
4,http://data.gesis.org/claimskg/claim_review/ef...,pete gallego cancel plan meet families sandy h...,2018-07-18,1,False,Roland Gutierrez,Roland Gutierrez errs saying Pete Gallego canc...,"2012 mass shooting,Carlos Uresti,Democrat,Face...",Pete Gallego,"Congress,Guns",politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,http://data.gesis.org/claimskg/claim_review/d7...,pedestrian kill fly fire hydrant,Unknown,3,True,Unknown,Fire Hydrant Death,"Associated Press,Encinitas, California,Fire Hy...",fire hydrant,"ASP Article, freakish fatalities",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/fire-plugged/,English
4448,http://data.gesis.org/claimskg/claim_review/b5...,unite industrialize country without universal ...,2019-04-30,3,True,Mark Pocan,Universal health care diagnosis is on the mark,"Affordable Care Act,Australia,Austria,Belarus,...",NaN,"Health Care,Medicaid,Medicare",politifact,http://www.politifact.com,http://www.politifact.com/wisconsin/statements...,English
4449,http://data.gesis.org/claimskg/claim_review/05...,yearold nigerian girl publicly flog engage pre...,Unknown,3,True,Unknown,Bariya Ibrahim Magazu Petition,"Abuja,Associated Press,Barbara “faith,Dallas M...",premarital sex,"ASP Article, Petitions",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/the-cane-mut...,English
4450,http://data.gesis.org/claimskg/claim_review/7d...,control house last two years malfunction sessi...,2010-10-18,3,True,William G. Batchelder,Republican state Rep. Bill Batchelder knocks D...,"Armond Budish,Democrat,PolitiFact,Republican",NaN,Government Efficiency,politifact,http://www.politifact.com,http://www.politifact.com/ohio/statements/2010...,English


### Etape 3 : Conversion des textes en valeurs numériques
 On transforme nos données de textes en valeurs numériques, en utilisant des LabelEncoder sur les colonnes qu'on a  sauf la colonne texte et la colonne keyword, dont on a essayé Tf-IDF pour les transformer. 

In [3]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer




#np.random.seed(500) #utilisé pour avoir le même résultat à chaque exécution 
#On crée des variable LabelEncoder qui vont servir à transférer nos données en valeurs numériques
l1=LabelEncoder()
l2=LabelEncoder()
l3=LabelEncoder()
l4=LabelEncoder()
l5=LabelEncoder()
l6=LabelEncoder()
l7=LabelEncoder()
l8=LabelEncoder()
l9=LabelEncoder()
l10=LabelEncoder()
l11=LabelEncoder()
l12=LabelEncoder()
l13=LabelEncoder()
df=df.applymap(str) #on transforme tous nous données en String car y'avais des entrées qui ont une combinaison du string et float
#On applique la mesure TF-IDF sur la colonne text et la colonne keywords
Tfidf_vect = TfidfVectorizer(max_features=1000,sublinear_tf=True, min_df=200, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidfx=Tfidf_vect.fit_transform(df['text'])
df1 = pd.DataFrame(tfidfx.toarray(), columns=Tfidf_vect.get_feature_names())
Tfidf_vect = TfidfVectorizer(max_features=30)
print(df['keywords'])
tfidfx=Tfidf_vect.fit_transform(df['keywords'])
df2 = pd.DataFrame(tfidfx.toarray(), columns=Tfidf_vect.get_feature_names())
print(df1)


#on transfère toutes les valeurs des colonnes qu'on va utiliser en valeurs numériques 
df['id']=l1.fit_transform(df['id'])
df['date']=l3.fit_transform(df['date'])
df['author']=l5.fit_transform(df['author'])
df['headline']=l6.fit_transform(df['headline'])
df['named_entities_claim']=l7.fit_transform(df['named_entities_claim'])
df['named_entities_article']=l8.fit_transform(df['named_entities_article'])
df['source']=l10.fit_transform(df['source'])
df['sourceURL']=l11.fit_transform(df['sourceURL'])
df['link']=l12.fit_transform(df['link'])
df['language']=l13.fit_transform(df['language'])

training1= pd.concat([df, df1], axis=1,join_axes=[df.index])
training=pd.concat([training1, df2], axis=1,join_axes=[df.index])








0                              Candidate Biography
1                                      Immigration
2       canada, marijuana, world news daily report
3                                            islam
4                                    Congress,Guns
                           ...                    
4447              ASP Article, freakish fatalities
4448                 Health Care,Medicaid,Medicare
4449                        ASP Article, Petitions
4450                         Government Efficiency
4451                                         Islam
Name: keywords, Length: 24452, dtype: object
       act  actually  administration  allow  america  american  americans  \
0      0.0       0.0             0.0    0.0      0.0       0.0        0.0   
1      0.0       0.0             0.0    0.0      0.0       0.0        0.0   
2      0.0       0.0             0.0    0.0      0.0       0.0        0.0   
3      0.0       0.0             0.0    0.0      0.0       0.0        0.0   
4      0.

C:\Users\Chahinez\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
C:\Users\Chahinez\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.


### Etape 4 : Dévision du jeu de données en données d'entraînement et données de test
On mélange le dataframe qu'on a et après on sélectionne 80% des données pour l'entraînement et 20% pour le test, ensuite on sélectionne les colonnes features dont on s'intéresse lors du classification, et on essaye plusieurs combinaisons de colonnes pour arriver à une meilleure accuracy (#TODO).

In [4]:
from sklearn.utils import shuffle
import numpy as np
#on mélange le dataframe
shuffled=shuffle(training1)
#on sélectionne 80% de notre jeu de données pour l'entrainement
training=shuffled.head(19562)
#on sélectionne 20% de notre jeu de données pour le test
testing=shuffled.tail(4890)
#les catégories de chaque entrée des données d'entrainement
value=training['truthRating']
print(training)
#les catégories réels de chaque entrée des données du test
realvalues=testing['truthRating']
#on élimine les colonnes des catégories (car le modèle va les prédir)
training=training.drop(columns='ratingName')
testing=testing.drop(columns='ratingName')
training=training.drop(columns='named_entities_claim')
testing=testing.drop(columns='named_entities_claim')
training=training.drop(columns='named_entities_article')
testing=testing.drop(columns='named_entities_article')
training=training.drop(columns='truthRating')
testing=testing.drop(columns='truthRating')
training=training.drop(columns='id')
testing=testing.drop(columns='id')
training=training.drop(columns='headline')
testing=testing.drop(columns='headline')
training=training.drop(columns='link')
testing=testing.drop(columns='link')
testing=testing.drop(columns='keywords')
training=training.drop(columns='keywords')
training=training.drop(columns='date')
testing=testing.drop(columns='date')
print(tfidfx)
print(value)
training=training.drop(columns='text')
testing=testing.drop(columns='text')

         id                                               text  date  \
9553  23116  germany release historic statement denounce unite  3737   
2632   7641  muslims disturb plan come grocery store go bar...  3048   
3831  19599         photograph halfblack halftabby chimera cat  3737   
6993   6601  new rasmussen poll people oppose sotomayor con...   379   
2305   1441  unite airlines flight attendant slap cry baby ...  2911   
...     ...                                                ...   ...   
4650  22476      barack plan call mandate fin small businesses   256   
9315  11704  send much interest china pay entire people lib...  1113   
1638  20185  rick perry never lose election remain person t...   961   
1627   6632  minnesota public school make learn arabic mand...  3737   
9500  11848  upwards women seek abortion decide abortion se...  1645   

     truthRating ratingName  author  headline  named_entities_claim  \
9553           2    MIXTURE    3716      5242                  9

### Etape 5 :  Entraînement et prédiction en utilisant des modèles de classification
On passe les features et targets des données d'entraînement à nos classifiers (#TODO tester plusieurs classifieurs) et après on lui laisse prédire les valeurs des données de test (soit vrai, faux ou mixture).

In [5]:
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection, naive_bayes, svm


Naive = naive_bayes.MultinomialNB()
Naive.fit(training,value)
result=Naive.predict(testing)
print(result)
print(accuracy_score(realvalues,result))


['2' '1' '1' ... '1' '2' '2']
0.49427402862985687
